參考資料:
https://huggingface.co/learn/nlp-course/chapter7/7?fw=pt

In [ ]:
!pip install datasets
!pip install peft
!pip install evaluate

In [ ]:
from datasets import load_dataset

# squad = load_dataset("squad")
squad = load_dataset("squad", split = "train[:20000]")

In [ ]:
squad_dataset = squad.train_test_split(test_size = 0.2)
squad_dataset

In [ ]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def preprocess_function_train(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length = 384,
        truncation = "only_second",
        stride = 128,
        return_overflowing_tokens = True,
        return_offsets_mapping = True,
        padding = "max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")

    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [ ]:
train_dataset = squad_dataset["train"].map(preprocess_function_train, batched = True, remove_columns = squad_dataset["train"].column_names)
train_dataset

In [ ]:
def preprocess_function_validation(examples):  # 有 labels
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length = 384,
        truncation="only_second",
        stride = 128,
        return_overflowing_tokens = True,
        return_offsets_mapping = True,
        padding = "max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")

    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    example_ids = []
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
val_dataset = squad_dataset["test"].map(preprocess_function_validation, batched = True, remove_columns = squad_dataset["test"].column_names)
val_dataset

In [ ]:
from tqdm.auto import tqdm
import collections
import evaluate
import numpy as np

metric = evaluate.load("squad")

n_best = 20
max_answer_length = 32

def compute_metrics(start_logits, end_logits, features, examples):

    # example_id 和 index 的對照表
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()  # 找出前20大的
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()  # 找出前20大的

            # 遍歷所有組合
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score(分數最高)
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]

    return metric.compute(predictions = predicted_answers, references = theoretical_answers)

In [ ]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

lora_config = LoraConfig(r = 16, lora_alpha = 32, lora_dropout = 0.1, target_modules = ["q", "v"], task_type = TaskType.QUESTION_ANS)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "T5_finetune_process",
    num_train_epochs = 5,
    learning_rate = 2e-5,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_strategy = "epoch",
    overwrite_output_dir = True
)

trainer = Trainer(
    model = peft_model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator
)

In [ ]:
trainer.train()

###Evaluate

In [ ]:
# 原先的模型
org_trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator
)

predictions, _, _ = org_trainer.predict(val_dataset)
start_logits, end_logits, _ = predictions
compute_metrics(start_logits, end_logits, val_dataset, squad_dataset["test"])

In [ ]:
# finetune 過後的模型
predictions, _, _ = trainer.predict(val_dataset)
start_logits, end_logits, _ = predictions  # predictions is a tuple
compute_metrics(start_logits, end_logits, val_dataset, squad_dataset["test"])

###Inference

In [ ]:
peft_model.save_pretrained("T5_LoRA_finetuned")

In [ ]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [ ]:
from transformers import pipeline
from peft import PeftModel

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
finetuned_model = PeftModel.from_pretrained(model, "T5_LoRA_finetuned")

# finetune過後的模型
question_answerer = pipeline("question-answering", model = finetuned_model, tokenizer = tokenizer)
question_answerer(question = question, context = context)

In [ ]:
# 原先的模型
question_answerer = pipeline("question-answering", model = model_name)
question_answerer(question = question, context = context)